In [59]:
import numpy as np
import sys
import os
sys.path.append(os.path.abspath('../'))
from scipy.io import loadmat
import loaders
from preprocessing.vsdi_preprocessing import clean_outliers,pca_ica,glm
from pathlib import Path
import matplotlib.pyplot as plt
import pandas as pd
import statsmodels.api as sm
import pickle
from tqdm import tqdm

# Fit GLMS

In [68]:
datapath = Path('/ceph/imaging1/davide/ATC_Data_preprocessed')
processed_datapath = Path('/scratch/dspalla/ATC_analysis')
glm_output = processed_datapath.joinpath('glm_results')
glm_output.mkdir(parents=True,exist_ok=True)


In [69]:
animals = ['A04','A06','A07','A08']
days = ['Day1','Day3','Day5','Day7']
predictor_labels = ['CS+','CS+_tr','sound','sound_trace','reward','lick']

for animal in animals:
    print(f'{animal}')
    timecourses = np.load(processed_datapath.joinpath(f'timecourses_{animal}.npy'))
    slice_start = 0
    for day in days:
        print('Loading data ...')
        atc1 = loadmat(datapath.joinpath(f'{animal}/{day}/ATC1.mat'))
        atc2 = loadmat(datapath.joinpath(f'{animal}/{day}/ATC2.mat'))
        b_data1 = loaders.extract_behavioural_data(atc1)
        b_data2 = loaders.extract_behavioural_data(atc2)
        X1 = make_design_matrix(b_data1)
        X2 = make_design_matrix(b_data2)
        
        X = np.concatenate((X1,X2))
        
        print(f'fitting {day} with shape: {X.shape}')
        
        X = sm.add_constant(X)
        print(f'fectching timecourses in range: {slice_start}-{slice_start+len(X)}')
        Y = timecourses.T[slice_start:slice_start+len(X)]
        
        for i,y in enumerate(Y.T):
            model = sm.GLM(y, X, family=sm.families.Gaussian())
            results = model.fit()
            with open(glm_output.joinpath(f'glm_results_{animal}_{day}_c{i}.pickle'),'wb') as pfile:
                pickle.dump(results,pfile)
        slice_start += len(X)-1
    
        
        
        

A04
Loading data ...
fitting Day1 with shape: (59999, 6)
fectching timecourses in range: 0-59999
Loading data ...
fitting Day3 with shape: (59999, 6)
fectching timecourses in range: 59998-119997
Loading data ...
fitting Day5 with shape: (59998, 6)
fectching timecourses in range: 119996-179994
Loading data ...
fitting Day7 with shape: (60000, 6)
fectching timecourses in range: 179993-239993
A06
Loading data ...
fitting Day1 with shape: (59998, 6)
fectching timecourses in range: 0-59998
Loading data ...
fitting Day3 with shape: (59999, 6)
fectching timecourses in range: 59997-119996
Loading data ...
fitting Day5 with shape: (59998, 6)
fectching timecourses in range: 119995-179993
Loading data ...
fitting Day7 with shape: (60000, 6)
fectching timecourses in range: 179992-239992
A07
Loading data ...
fitting Day1 with shape: (60000, 6)
fectching timecourses in range: 0-60000
Loading data ...
fitting Day3 with shape: (59998, 6)
fectching timecourses in range: 59999-119997
Loading data ...
fi

In [49]:
?results

Type:            GLMResultsWrapper
String form:     <statsmodels.genmod.generalized_linear_model.GLMResultsWrapper object at 0x14ce24ada440>
File:            /scratch/dspalla/mambaforge/lib/python3.10/site-packages/statsmodels/genmod/generalized_linear_model.py
Docstring:      
Class to contain GLM results.

GLMResults inherits from statsmodels.LikelihoodModelResults

Attributes
----------
df_model : float
    See GLM.df_model
df_resid : float
    See GLM.df_resid
fit_history : dict
    Contains information about the iterations. Its keys are `iterations`,
    `deviance` and `params`.
model : class instance
    Pointer to GLM model instance that called fit.
nobs : float
    The number of observations n.
normalized_cov_params : ndarray
    See GLM docstring
params : ndarray
    The coefficients of the fitted model.  Note that interpretation
    of the coefficients often depends on the distribution family and the
    data.
pvalues : ndarray
    The two-tailed p-values for the parameters.


In [ ]:
# LOAD TIMECOURSES
# SPLIT EACH DAY USING ATC1 info
# FIT GLM ON EACH COMPONENT ON EACH DAY
# SAVE RESULTS